# EEG data loading and pre-processing 

### Packages import

Packages are Python librairies, i.e. files that contain pre-written functions. We need to import the packages before using the functions that they include. For our purpose (MEG/EEG data analysis), most of the functions will be included in the MNE package, but we may have to use other packages too.

In [15]:
! pip install mne 
# I know we said this is bad practice, but let's do it anyways so we don't have to open any terminal ;)
! pip install scikit-learn --user

import mne # Here we import mne, the package that will contain most of the function that we will use today.
from mne.datasets.brainstorm import bst_raw # It is possible to import functions individually. This is helpful since it 
                                            # saves time, memory, and makes the calls to the function easier.
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
%matplotlib notebook
# ^ This last line lets us use the interactive plots in Jupyter

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Loading the data

In [16]:
PARTICIPANTS = ['01', '04', '05', '06', '07', '09', '11', '12', '13', '14']

In [18]:
data_path = r'C:\Users\Dell\Jupyter\MNE_python'
raw = {}
#Load data for each of the 10 participants
for p in PARTICIPANTS:
    raw_fname = data_path + '\P' + p + '-raw.fif'
    raw[p] = mne.io.read_raw_fif(raw_fname, preload=True)
    

Opening raw data file C:\Users\Dell\Jupyter\MNE_python\P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
Opening raw data file C:\Users\Dell\Jupyter\MNE_python\P04-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2480032 =      0.000 ...  4843.812 secs
Ready.
Reading 0 ... 2480032  =      0.000 ...  4843.812 secs...
Opening raw data file C:\Users\Dell\Jupyter\MNE_python\P05-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2521101 =      0.000 ...  4924.025 secs
Ready.
Reading 0 ... 2521101  =      0.000 ...  4924.025 secs...
Opening raw data file C:\Users\Dell\Jupyter\MNE_python\P06-raw.fif...
Isotrak not found
    Read a total of 1 pr

In [14]:
#Remove EXG5 and EXG6 from participant 9 and higher
raw['13'].info['ch_names']
raw[p].drop_channels(['EXG5-EXG6'])

['Fp1',
 'AF7',
 'AF3',
 'F1',
 'F3',
 'F5',
 'F7',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'C1',
 'C3',
 'C5',
 'T7',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'P1',
 'P3',
 'P5',
 'P7',
 'P9',
 'PO7',
 'PO3',
 'O1',
 'Iz',
 'Oz',
 'POz',
 'Pz',
 'CPz',
 'Fpz',
 'Fp2',
 'AF8',
 'AF4',
 'AFz',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT8',
 'FC6',
 'FC4',
 'FC2',
 'FCz',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP8',
 'CP6',
 'CP4',
 'CP2',
 'P2',
 'P4',
 'P6',
 'P8',
 'P10',
 'PO8',
 'PO4',
 'O2',
 'EXG1',
 'EXG2',
 'EXG3',
 'EXG4',
 'EXG5',
 'EXG6',
 'STI 014']

In [1]:
# Epoching for each condition

#events = mne.find_events(raw['04'])


events = mne.find_events(raw[p])
events_modif_cond = events.copy()
events_modif = events.copy()
for e in events_modif_cond:
    if int(str(e[2])) == 1111:
        e[2] = 0
for e in events_modif_cond:
    if int(str(e[2])) == 2001:
        e[2] = 0
        
for e in events_modif_cond:
    if int(str(e[2])[-1]) == 1:
        e[2] = 1
    elif int(str(e[2])[-1]) == 2:
        e[2] = 2
    elif int(str(e[2])[-1]) == 3:
        e[2] = 3

        #int(str(events[0][2])[-1])
event_id_cond = {'Percept': 1, 'Imagine_cued': 2}

event_id = {'S1_p': 11, 'S1_i': 12,
            'S2_p': 21, 'S2_i': 22, 
            'S3_p': 31, 'S3_i': 32,
            'S4_p': 41, 'S4_i': 42,
            'S11_p': 111, 'S11_i': 112,
            'S12_p': 121, 'S12_i': 122, 
            'S13_p': 131, 'S13_i': 132,
            'S14_p': 141, 'S14_i': 142,
            'S21_p': 211, 'S21_i': 212,
            'S22_p': 221, 'S22_i': 222, 
            'S23_p': 231, 'S23_i': 232,
            'S24_p': 241, 'S24_i': 242}

events_modif_cond

NameError: name 'mne' is not defined

In [13]:
##OVERALL PREPROCESSING FOR ALL PARTICIPANTS


for p in PARTICIPANTS:

#Filtering
    raw[p].filter(0.5, 90., fir_design="firwin") # This method applies a bandpass filter to the data in "raw".
                                           # Here, we keep only frequencies between 0.5 and 150Hz, the rest will be attenuated.
    
    raw[p].notch_filter([60, 120], filter_length='auto',phase='zero', fir_design="firwin") # The notch filter will only
                                            # remove certain frequencies. Since in North America the power lines oscillates
                                            # at 60Hz, we will remove this frequency and it's first harmonic, 120Hz.
        
#ICA

    ica = mne.preprocessing.ICA(n_components=20, random_state=0).fit(raw[p], decim=3)

    eog_epochs = create_eog_epochs(raw[p], ch_name='EXG2')
    eog_inds, eog_scores = ica.find_bads_eog(eog_epochs, ch_name='EXG2')
    #ica.plot_scores(eog_scores, eog_inds);
    #ica.plot_properties(eog_epochs, picks=eog_inds, psd_args={'fmax': 35.},
                    #image_args={'sigma': 1.});

    ica.exclude = eog_inds
    ica.apply(raw[p]) # Here we replace the old raw signal by the ICA recomposition

#Epoching


    tmin, tmax = -0.5, 6.8 # Here we define the amount of time we want to keep before (tmin) and after (tmax) the event.
                       
    
    baseline = (-0.5, -0.1) # When epoching, we often want to use some kind of baseline correction. Here, we define that our baseline
                       # starts at the beginning of the epoch and goes until the stimulus occurence (t=0)
    
    reject = {'eeg': 40e-6} # The Epochs() function allows to automatically reject events in which one or more channels exceeds a
                        # threshold value. We can set different values for different types of channels (mag, grad, eeg etc...).
                        # This is useful because even a meticulous pre-processing may leave some noisy segments in the data, and
                        # this method only rejects the contaminated trials while keeping the others intact.
            
    picks = mne.pick_types(raw[p].info, eeg=True)
                       #eeg=False, eog=False) # We choose to analyse only the magnetometers ('mag')
    
    epochs = mne.Epochs(raw[p], events=events_modif_cond, event_id=event_id_cond, tmin=tmin,
                        tmax=tmax, baseline=baseline, reject=None)
    epochs.save(data_path + '\\' + 'epoched_data_cond' + p +'.fif.gz', overwrite=True)
    #epochs.save(p + '.fif.gz')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3379 samples (6.600 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3379 samples (6.600 sec)

Fitting ICA to data using 61 channels (please be patient,

<ipython-input-13-fa34ba83ce72>:46: RuntimeWarning: This filename (C:\Users\Dell\Jupyter\MNE_python\epoched_data_cond01.fif.gz) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(data_path + '\\' + 'epoched_data_cond' + p +'.fif.gz', overwrite=True)


0 bad epochs dropped
Loading data for 1 events and 3739 original time points ...
Loading data for 120 events and 3739 original time points ...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3379 samples (6.600 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.5

<ipython-input-13-fa34ba83ce72>:46: RuntimeWarning: This filename (C:\Users\Dell\Jupyter\MNE_python\epoched_data_cond04.fif.gz) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(data_path + '\\' + 'epoched_data_cond' + p +'.fif.gz', overwrite=True)


0 bad epochs dropped
Loading data for 1 events and 3739 original time points ...
Loading data for 120 events and 3739 original time points ...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3379 samples (6.600 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.5

...

0 bad epochs dropped
Loading data for 1 events and 3739 original time points ...
Loading data for 120 events and 3739 original time points ...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3379 samples (6.600 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.5

<ipython-input-13-fa34ba83ce72>:46: RuntimeWarning: This filename (C:\Users\Dell\Jupyter\MNE_python\epoched_data_cond13.fif.gz) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(data_path + '\\' + 'epoched_data_cond' + p +'.fif.gz', overwrite=True)


0 bad epochs dropped
Loading data for 1 events and 3739 original time points ...
Loading data for 120 events and 3739 original time points ...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3379 samples (6.600 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.5

<ipython-input-13-fa34ba83ce72>:46: RuntimeWarning: This filename (C:\Users\Dell\Jupyter\MNE_python\epoched_data_cond14.fif.gz) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(data_path + '\\' + 'epoched_data_cond' + p +'.fif.gz', overwrite=True)


0 bad epochs dropped
Loading data for 1 events and 3739 original time points ...
Loading data for 120 events and 3739 original time points ...


### MNE data objects

The MNE data objects contains the data + some information (sampling frequency, number of channels etc...). It is very convenient since most MNE functions will be applied directly on this object, and you can access everything in the same place. 

In [ ]:
raw['01'].info # For example, you can use the method "info" to look at the informations about your data
#raw['01'].info.get('ch_names')
#raw.check_trial_events()

In [ ]:
raw.info.get('sfreq') # Or you can access each field separately by using get() (here, we ask for the sampling frequency)

In [ ]:
raw.plot(); # You can also plot the timeseries of your data and scroll to explore them visually.
           # The plot method can take many arguments to help you plot the data the way you want.
           # Any transformation commanded by these arguments (e.g. filtering) will ONLY apply to the plotted data, 
           # the content of "raw" will be left untouched.

In [ ]:
# Let's have a quick look at the spectrum of our data
raw.plot_psd(tmax=60., average=False);

### Filtering

Before doing anything with our data, we usually want to filter them in order to remove specific frequency bands.
1. We want to remove frequencies that are either too low or too high to be meaningful to us
    -> Band-pass filter
2. We want to remove the line noise (i.e. noise generated by the alternative current used to power electric devices)
    -> Notch filter

In [ ]:


raw['04'].filter(0.5, 90., fir_design="firwin") # This method applies a bandpass filter to the data in "raw".
                                           # Here, we keep only frequencies between 0.5 and 150Hz, the rest will be attenuated.
    
raw['04'].notch_filter([60, 120], filter_length='auto',phase='zero', fir_design="firwin") # The notch filter will only
                                            # remove certain frequencies. Since in North America the power lines oscillates
                                            # at 60Hz, we will remove this frequency and it's first harmonic, 120Hz.

In [ ]:
# Plot a new spectrum to see how the signal was modified
raw['04'].plot_psd(tmax=60., average=False);

### ICA decomposition

The next classical step to MEG/EEG pre-processing is to run an Independent Component Analysis (ICA) in order to detect and remove occular and cardiac artifacts from our signal.

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=0).fit(raw['04'], decim=3) # We compute an ICA object from the data.
# The n_components argument defines the number of components that we wish to obtain.
# It is also possible to state that 0<n_components<1, in which case it will define the proportion of variance that 
# we want to be explained by the N components. 

In [ ]:
ica.plot_sources(raw['04']); # We can plot our components and explore them visually to get 
                      # an idea of which artifact was captured by which component. What do we see ?

In [ ]:
ica.plot_components(outlines='skirt')

Looking at the above plot, we can guess that the last component (ICA019) captures cardiac activity. ICA001 looks like it has captured ocular blinks.
However, this is kind of arbitrary. Let's confirm our guesses with an automatic method. Hopefully, this dataset has an ECG and EOG channels that will help us find the artifactual components. If it is not the case, this method will still work although it may make more mistakes. 

In [ ]:
# Now, let's do the same for ocular blinks. Note that two things have changed :
# - The functions used to detect artifacts are now called "eog" instead of "ecg"
# - The channel to which we correlate our components is now EEG058, the EOG channel
eog_epochs = create_eog_epochs(raw, ch_name='EXG2')
eog_inds, eog_scores = ica.find_bads_eog(eog_epochs, ch_name='EXG2')
ica.plot_scores(eog_scores, eog_inds);
ica.plot_properties(eog_epochs, picks=eog_inds, psd_args={'fmax': 35.},
                    image_args={'sigma': 1.});

In [ ]:
# Finally, we can recompose our signal while excluding the artifactual components : 

#ica.exclude = [1, 19] # We can also index the components individually, useful if we don't agree with the automatic detection

#ica.exclude = ecg_inds # Exclude cardiac (ECG) or blinks (EOG) components 
#ica.apply(raw) # Here we replace the old raw signal by the ICA recomposition

ica.exclude = eog_inds
ica.apply(raw) # Here we replace the old raw signal by the ICA recomposition


raw.plot(); # Plot the clean signal.

### Saving datasets

In [ ]:
# Now that our pre-processing is done, it is time to save our clean dataset

filename = 'clean_dataset'
raw.save(data_path + '\\' + filename + '_raw.fif.gz', overwrite=True) # May be necessary to switch between '\\' and '/' depending on your OS

In [ ]:
events = mne.find_events(raw['04'])
events_modif = events.copy()

event_id = {'S1_p': 11, 'S1_i': 12,
            'S2_p': 21, 'S2_i': 22, 
            'S3_p': 31, 'S3_i': 32,
            'S4_p': 41, 'S4_i': 42,
            'S11_p': 111, 'S11_i': 112,
            'S12_p': 121, 'S12_i': 122, 
            'S13_p': 131, 'S13_i': 132,
            'S14_p': 141, 'S14_i': 142,
            'S21_p': 211, 'S21_i': 212,
            'S22_p': 221, 'S22_i': 222, 
            'S23_p': 231, 'S23_i': 232,
            'S24_p': 241, 'S24_i': 242}

In [ ]:
mne.viz.plot_events(events, raw.info['sfreq'], raw.first_samp);
                    #event_id=event_id);

In [ ]:
tmin, tmax = -0.5, 6.8 # Here we define the amount of time we want to keep before (tmin) and after (tmax) the event.
                       # Thus, we will split our signal into 0.7s chunks.
    
baseline = (-0.5, -0.1) # When epoching, we often want to use some kind of baseline correction. Here, we define that our baseline
                       # starts at the beginning of the epoch and goes until the stimulus occurence (t=0)
    
reject = {'eeg': 40e-6} # The Epochs() function allows to automatically reject events in which one or more channels exceeds a
                        # threshold value. We can set different values for different types of channels (mag, grad, eeg etc...).
                        # This is useful because even a meticulous pre-processing may leave some noisy segments in the data, and
                        # this method only rejects the contaminated trials while keeping the others intact.
            
picks = mne.pick_types(raw['04'].info, eeg=True)
                       #eeg=False, eog=False) # We choose to analyse only the magnetometers ('mag')

epochs = mne.Epochs(raw['04'], events=events_modif, event_id=event_id, tmin=tmin,
                    tmax=tmax, baseline=baseline, reject=None)
                    #, picks=picks) # This function splits the signal into epochs that
                                                                             # can be accessed separately by calling epochs[condition]

#epochs.drop_bad() # Here, we drop the epochs that were marked as bad using the "reject" parameter. 
                  # The dropped epochs will not be used for further analysis.

In [ ]:
evoked = epochs['Percept'].average()

In [ ]:
epochs['Percept'].plot_psd_topomap(); # You can also plot the topographies of frequency bands.
epochs['Imagine_cued'].plot_psd_topomap();  # Let's compare both conditions !

In [ ]:
percept_epochs = epochs['Percept'].average()
imagined_epochs = epochs['Imagine_cued'].average()

In [ ]:
imagined_epochs.plot_joint();

In [ ]:
mne.combine_evoked((percept_epochs, imagined_epochs), weights=(.5, -.5)).plot_joint(times=.35);

In [ ]:
mne.viz.plot_compare_evokeds({"percept": percept_epochs, "imagined": imagined_epochs},
                             picks=[22], truncate_yaxis=True);

In [ ]:
data_percept = epochs['Percept'].get_data()
data_imagined = epochs['Imagine_cued'].get_data()
hurst_sensors_p = []
hurst_sensors_i = []
#first sensor, all events  (event, sensor, timestamp)
for s in range(64):
    hurst_sensors_p.append(hurst(data_percept[:, s, :]))
    hurst_sensors_i.append(hurst(data_imagined[:, s, :]))


